In [1]:
import cudf as pd
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
adv_data = pd.read_csv('data/adv_data.csv')
event_data = pd.read_csv('data/event_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
adv_data.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,<NA>
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,<NA>
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,<NA>
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,<NA>
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [4]:
event_data.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,+79140092454,show,2023-10-05 12:03:04.125806,<NA>,<NA>,<NA>,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,+79125904946,show,2023-05-16 10:16:54.554523,<NA>,<NA>,<NA>,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,+79184556300,show,2023-12-23 22:46:44.066599,<NA>,<NA>,<NA>,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,+79119287528,show,2023-07-30 05:25:46.285514,<NA>,<NA>,<NA>,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,+79119287528,click,2023-07-30 05:25:46.285514,<NA>,2023-07-30 05:42:59.014970,2.18,USD,loan


In [3]:
users_data.drop('delete_dtm', inplace=True, axis=1)

In [4]:
event_phones = event_data['user_phone'].to_arrow()

In [5]:
np_test = np.zeros_like(event_phones)

In [6]:
for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')

In [7]:
event_data['user_phone_edited'] = np_test

In [8]:
event_data.drop('user_phone', axis=1, inplace=True)

In [9]:
user_phones = users_data['user_phone'].to_arrow()

In [10]:
np_test = np.zeros_like(user_phones)

In [11]:
for i, val in enumerate(user_phones):
    np_test[i] = str(val).strip('+')

In [12]:
users_data['user_phone_edited'] = np_test

In [13]:
users_data.drop('user_phone', axis=1, inplace=True)

In [14]:
adv_data_plus_user_data = adv_data.merge(users_data, how='inner', on='user_id')

In [15]:
del adv_data, users_data

In [16]:
all_data = adv_data_plus_user_data.merge(event_data, how='inner', on='user_phone_edited')

In [17]:
del event_data

In [18]:
all_data.drop(['id', 'user_id'], inplace=True, axis=1)

In [59]:
# chunk_size = 150_000
# start_no = 0
# end_no = chunk_size
# chunk_amount = int(len(all_data) / chunk_size) + 1

In [61]:
# for i in range(chunk_amount):
#     all_data.iloc[start_no:end_no].to_csv(f'all_data/all_data_{i}.csv')
#     start_no += chunk_size
#     end_no += chunk_size